In [1]:
from transformers import GPT2Tokenizer, TFGPT2LMHeadModel
import tensorflow as tf
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = TFGPT2LMHeadModel.from_pretrained('gpt2',use_cache=False)

All model checkpoint weights were used when initializing TFGPT2LMHeadModel.

All the weights of TFGPT2LMHeadModel were initialized from the model checkpoint at gpt2.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


In [2]:
ids = tokenizer.encode("who is a member of the National Security Council,and is a member of",return_tensors = 'tf')

In [3]:
import numpy as np 



NameError: name 'concrete_function' is not defined

In [4]:
x = list()
for i in ["She was really good girl","She was really good girl","She was really good girl"]:
    ids = tokenizer.encode(i,return_tensors = 'tf')
    x.append(ids[0])

In [5]:
np.array(x)

array([[3347,  373, 1107,  922, 2576],
       [3347,  373, 1107,  922, 2576],
       [3347,  373, 1107,  922, 2576]])

In [6]:
# import numpy as np 


NameError: name 'concrete_function' is not defined

In [7]:
callable = tf.function(model.call)
concrete_function = callable.get_concrete_function(tf.TensorSpec([None,None], tf.int32, name="input_ids"))

In [9]:
logits = concrete_function(np.array(x))[0][:,-1]

In [8]:
tokenizer.decode([np.argmax(concrete_function(np.array(x))[0][0][-1])])

'.'

In [67]:
@tf.function
def Top_k(logits,temperature=0.9,k=50):
    logits = tf.divide(logits,temperature)
#    Take top 50 and then divide the logits by its temperature so we can make low probalabe values even more probabale and less probable values even less probable
    values,_=tf.math.top_k(logits, k=50, sorted=True,name = "top50") #selecting Top 50 logits
#     select the minimum value from each axis and divide with temperature
    minimum_values = values[:,-1][:,tf.newaxis]
#     Now make all the values below this -1e*10 in the logits this happens by broadcasting 
    return tf.where(logits<minimum_values,tf.ones_like(logits, dtype=logits.dtype) * -1e10,logits)

In [68]:
@tf.function
def Top_p(logits, p=0.95):
    """Nucleus sampling"""
    batch = 3
    sorted_logits = tf.sort(logits, direction='DESCENDING', axis=-1)
    cumulative_probs = tf.cumsum(tf.nn.softmax(sorted_logits, axis=-1), axis=-1)
    indices = tf.stack([
        tf.range(0, batch),
        # number of indices to include
        tf.maximum(tf.reduce_sum(tf.cast(cumulative_probs <= p, tf.int32), axis=-1) - 1, 0),
    ], axis=-1)
    min_values = tf.gather_nd(sorted_logits, indices)[:,tf.newaxis]
    return tf.where(
        logits < min_values,
        tf.ones_like(logits) * -1e10,
        logits,
    )    

In [74]:
@tf.function
def get_output(input_ids,len,top_k,top_p,temp):
    tokens = input_ids
    for _ in tf.range(len):
        outputs = tf.random.categorical(Top_p(Top_k(concrete_function(tokens)[0][:,-1],temp,top_k),temp),num_samples = 1)
        tokens = tf.concat([tokens,tf.cast(outputs,dtype = tf.int32)],1)
    return tokens

In [118]:
function = get_output.get_concrete_function(tf.TensorSpec([None,None], tf.int32, name="input_ids"),tf.TensorSpec((),tf.int32,name = "len"),tf.TensorSpec((),tf.int32,name="top_k"),tf.TensorSpec((),tf.float32,name="top_p"),tf.TensorSpec((),tf.float32,name="temp"))

In [120]:
function(np.array(x),tf.constant(10),tf.constant(40),tf.constant(.95),tf.constant(.9))

<tf.Tensor: shape=(3, 15), dtype=int32, numpy=
array([[ 1858,  5615,   257,  1468, 16365,   290,   257,  1862,  1200,
           11,   351,   262,  2802,   338,  1735],
       [ 1858,  5615,   257,  1468, 16365,   290,   257,  1862, 16365,
           13,  1119,   550,   262,   976,  2156],
       [ 1858,  5615,   257,  1468, 16365,   319,   428,  8598,    11,
          290,   340,   373,   845,  1468,    13]])>

In [121]:
tf.saved_model.save(model,'Top_sentences_2',signatures = function)

INFO:tensorflow:Assets written to: Top_sentences_2\assets


In [122]:
model_ = tf.saved_model.load('Top_sentences_2/001')

In [123]:
model_.signatures['serving_default'](input_ids = tf.constant(np.array(x)),len=tf.constant(10),top_p=tf.constant(.9),top_k =tf.constant(40),temp = tf.constant(.9))['output_0']

<tf.Tensor: shape=(3, 15), dtype=int32, numpy=
array([[ 1858,  5615,   257,  1468, 16365,   508,   373,   262, 16365,
          286,   262,  1755,   290,   508,   973],
       [ 1858,  5615,   257,  1468, 16365,   290,   257,   922,  1862,
          582,    13,   383,  1468, 16365,   290],
       [ 1858,  5615,   257,  1468, 16365,    11,   508,  1718,   262,
         1438,   286,   705,   464,  7542,  4458]])>

In [98]:
x = list()
for i in ["There lived a old witch","There lived a old witch","There lived a old witch"]:
    ids = tokenizer.encode(i,return_tensors = 'tf')
    x.append(ids[0])

In [656]:
tokens = tf.constant(np.array(x),dtype = tf.int32)
for i in range(10):
    outputs = tf.random.categorical(Top_p(Top_k(concrete_function(tokens)[0][:,-1]),0.95),num_samples = 1)
    tokens = tf.concat([tokens,tf.cast(outputs,dtype = tf.int32)],1)

In [657]:
for i in tokens:    
    print(tokenizer.decode(i))

There lived a old witch and a man named Ralhaut who were
There lived a old witch and a young young man. The story of the
There lived a old witch, a woman of great intelligence and good taste,


In [650]:
callable = tf.function(model.call)

In [651]:
concrete_function = callable.get_concrete_function(tf.TensorSpec([None,None], tf.int32, name="input_ids"))

In [887]:
get_output(np.array(x)[0])

In [888]:
function = get_output.get_concrete_function(tf.TensorSpec([None], tf.int32, name="input_ids"))

In [828]:
outputs = function(np.array(x))

In [829]:
print(tokenizer.decode(outputs[0]))

There lived a old witch who lived in a castle, and a man of


In [890]:
tokenizer.decode(outputs[2])

'There lived a old witch, who died of her illness. And what the'

In [831]:
tf.saved_model.save(model,'Top_sentences_1',signatures = function)

INFO:tensorflow:Assets written to: Top_sentences_1\assets


In [832]:
model_ = tf.saved_model.load('Top_sentences_1')

In [835]:
words=model_.signatures['serving_default'](input_ids=tf.constant(np.array(x)))['output_0']

In [836]:
for i in words:
    print(tokenizer.decode(i))

There lived a old witch with her husband who used to tell her about her
There lived a old witch that lived in the house of the witches and the
There lived a old witch and her husband who would be executed for witchcraft.


In [1]:
import json

In [124]:
import requests
server_url = "http://localhost:8501/v1/models/TypeAhead:predict"

In [125]:
import numpy as np
np.array(x).shape

(3, 5)

In [126]:
input_data_json = json.dumps({"instances":[{"len":5,"temp":0.9,"input_ids":np.array(x).tolist(),"top_k":40,"top_p":0.95}]})

In [127]:
input_data_json

'{"instances": [{"len": 5, "temp": 0.9, "input_ids": [[1858, 5615, 257, 1468, 16365], [1858, 5615, 257, 1468, 16365], [1858, 5615, 257, 1468, 16365]], "top_k": 40, "top_p": 0.95}]}'

In [128]:
response = requests.post(server_url,data = input_data_json)

In [129]:
response.text

'{\n    "error": "The second input must be a scalar, but it has shape [1]\\n\\t [[{{node while}}]]"\n}'

In [42]:
tf.constant(5).as_list()

AttributeError: 'tensorflow.python.framework.ops.EagerTensor' object has no attribute 'as_list'

In [916]:
for i in np.array(response.json()['predictions']):
    print(tokenizer.decode(i))

KeyError: 'predictions'

In [186]:
x = list()
for i in ["Once upon a time there lived a king without fear whatsoever","Once upon a time there lived a king without fear whatsoever","Once upon a time there lived a king without fear whatsoever"]:
    ids = tokenizer.encode(i,return_tensors = 'tf')
    x.append(ids)

In [188]:
np.array(x)

array([[[ 7454,  2402,   257,   640,   612,  5615,   257,  5822,  1231,
          3252, 16014]],

       [[ 7454,  2402,   257,   640,   612,  5615,   257,  5822,  1231,
          3252, 16014]],

       [[ 7454,  2402,   257,   640,   612,  5615,   257,  5822,  1231,
          3252, 16014]]])

In [109]:
concrete_function(np.array(x))[0][0][-1]

<tf.Tensor: shape=(50257,), dtype=float32, numpy=
array([-74.626976, -75.38203 , -82.51564 , ..., -87.272064, -83.40596 ,
       -77.62991 ], dtype=float32)>

In [88]:
model(np.array(x[0]))

(<tf.Tensor: shape=(1, 11, 50257), dtype=float32, numpy=
 array([[[ -34.564377,  -34.408035,  -38.307804, ...,  -41.699547,
           -39.78006 ,  -35.05196 ],
         [ -84.725494,  -82.93255 ,  -87.01647 , ...,  -91.66669 ,
           -86.23538 ,  -84.70934 ],
         [-109.07983 , -105.72582 , -109.91147 , ..., -114.284645,
          -107.693275, -105.36123 ],
         ...,
         [ -91.777664,  -92.0373  ,  -97.055466, ...,  -95.54817 ,
           -95.6129  ,  -94.27099 ],
         [ -66.723076,  -67.995895,  -74.756294, ...,  -77.81113 ,
           -75.29102 ,  -70.35248 ],
         [ -74.62697 ,  -75.38202 ,  -82.51563 , ...,  -87.27206 ,
           -83.405945,  -77.629906]]], dtype=float32)>,)

In [178]:
tokenizer.decode([np.argmax(model_.signatures['serving_default'](ids[0])['output_0'])])

'!'

In [179]:
tokenizer.decode(model_.signatures['serving_default'](ids[0])['output_1'])

',. of; and for in:." that to who\n but—,"... as the about at ; from or (!… on [ — which-- against.... among except. because upon - a before with" -- – when over? save'

In [69]:
tf.nn.softmax(tf.math.top_k(model(ids)[0][0][-1], k=50, sorted=True,name = "top50")[0])

<tf.Tensor: shape=(), dtype=float32, numpy=0.8558431>

<tf.Tensor: shape=(50257,), dtype=float32, numpy=
array([3.9849447e-06, 3.8622125e-04, 3.7329280e-06, ..., 5.2050410e-08,
       1.8702770e-09, 1.6319476e-05], dtype=float32)>